In [1]:
import numpy as np
import pandas as pd

import geopy.distance

from utils.PreProcessor import PreProcessor

# Loading Datasets

In [2]:
%time preprocessor = PreProcessor(sample_num = 1, preprocess_all_raw_data = False)

**************************************
Loading PreProcessed Datasets
Loaded AirRouteDatasets
Loaded Domestic Passenger Data
Loaded Indian Railways Data
Loaded Socio-Economic Data
Loaded Monument Visitors Data
**************************************
CPU times: user 21.9 ms, sys: 6.35 ms, total: 28.3 ms
Wall time: 26.1 ms


In [3]:
preprocessor.city_mapping.head()

,City,AirRouteData_AirportCode,DomesticPassengerTraffic_City,EconomicData_District,TourismData_City,StationCodeData_District,Airport_City_Coords
0,Ahmedabad,AMD,AHMEDABAD,NaN,"None, closest Vadodara (70 miles)",Ahmadabad,"23.073702560470423, 72.62711816841471"
1,Bengaluru,BLR,BENGALURU,BANGALORE URBAN,Bangalore,Bangalore Urban,"13.199253589160458, 77.70610499457639"
2,Mumbai,BOM,MUMBAI,MUMBAI,Mumbai,Greater Bombay,"19.099282513860523, 72.87502185672862"
3,Pune,PNQ,PUNE,PUNE,"None, closest Mumbai (95 miles)",Pune,"18.57956672029852, 73.90948542603986"
4,Chennai,MAA,CHENNAI,CHENNAI,Chennai,Chennai,"12.994341981037747, 80.17159635853399"


In [4]:
preprocessor.all_network_data.head()

,From,To,Distance,Time,Cheapest Price,Aircraft Type,Number of Flights
0,AMD,IXC,583,105,59,Narrow Body,5.0
1,AMD,DED,598,110,103,Narrow Body,1.0
2,AMD,DEL,469,95,48,Narrow Body,24.0
3,AMD,JAI,326,80,53,Narrow Body,1.0
4,AMD,LKO,575,105,69,Narrow Body,4.0


In [5]:
preprocessor.network_data.head()

,From,To,Dummy
0,COK,BLR,-1
1,COK,DEL,-1
2,MAA,BLR,-1
3,MAA,HYD,-1
4,MAA,GAU,-1


In [6]:
preprocessor.total_domestic_data.head()

,Route,FROM,TO,PASSENGERS,FREIGHT,MAIL,DAYS
0,AGARTALA-AHMEDABAD,AGARTALA,AHMEDABAD,3038.0,52.357,0.0,365
1,AGARTALA-AIZAWL,AGARTALA,AIZAWL,5780.0,0.087,0.0,365
2,AGARTALA-BENGALURU,AGARTALA,BENGALURU,44195.0,367.947,0.0,365
3,AGARTALA-BHUBANESWAR,AGARTALA,BHUBANESWAR,18.0,0.000,0.0,61
4,AGARTALA-CHENNAI,AGARTALA,CHENNAI,12767.0,0.000,0.0,304


In [7]:
print(len(preprocessor.city_to_city_train_dict))
print(len(preprocessor.city_to_city_train_dict['Greater Bombay']['Pune']))
preprocessor.city_to_city_train_dict['Greater Bombay']['Pune']

20
54


[{'duration': 184.38666666671634,
  'train_num': '11017',
  'third_ac': 1,
  'chair_car': 0,
  'first_class': 0,
  'sleeper': 1,
  'second_ac': 1,
  'type': 'Exp',
  'first_ac': 0},
 {'duration': 209.31081081181765,
  'train_num': '11019',
  'third_ac': 1,
  'chair_car': 0,
  'first_class': 0,
  'sleeper': 1,
  'second_ac': 1,
  'type': 'Exp',
  'first_ac': 0},
 {'duration': 202.9632432423532,
  'train_num': '11023',
  'third_ac': 1,
  'chair_car': 0,
  'first_class': 0,
  'sleeper': 1,
  'second_ac': 1,
  'type': 'Exp',
  'first_ac': 0},
 {'duration': 185.25396825373173,
  'train_num': '11025',
  'third_ac': 0,
  'chair_car': 1,
  'first_class': 0,
  'sleeper': 0,
  'second_ac': 0,
  'type': 'Exp',
  'first_ac': 0},
 {'duration': 200.63416988402605,
  'train_num': '11027',
  'third_ac': 1,
  'chair_car': 0,
  'first_class': 0,
  'sleeper': 1,
  'second_ac': 1,
  'type': 'Mail',
  'first_ac': 0},
 {'duration': 199.4989189170301,
  'train_num': '11029',
  'third_ac': 0,
  'chair_car': 1

In [8]:
preprocessor.economic_data.head()

,State,District,Year,GDP
0,AndhraPradesh,ADILABAD,1999-00,3463.28
1,AndhraPradesh,ADILABAD,2000-01,3742.69
2,AndhraPradesh,ADILABAD,2001-02,4209.40
3,AndhraPradesh,ADILABAD,2002-03,4099.29
4,AndhraPradesh,ADILABAD,2003-04,4781.85


In [9]:
preprocessor.monument_visitors_data.head()

,City,NumMonuments2020,NumMonuments2022,Domestic2019-20,Domestic2020-21,Domestic2021-22,Foreign2019-20,Foreign2020-21,Foreign2021-22,Domestic2018-19,Foreign2018-19
0,Agra,9.0,9.0,7632914.0,2061695.0,5209587.0,1394635.0,14099.0,60086.0,9525738.0,1826587.0
1,Aurangabad,6.0,6.0,3545221.0,555097.0,1293929.0,56402.0,409.0,2673.0,3764094.0,73773.0
2,Bangalore,4.0,4.0,1420289.0,336796.0,529404.0,34483.0,1069.0,3348.0,1566190.0,41835.0
3,Bhopal,14.0,11.0,1953517.0,634016.0,1271045.0,61743.0,496.0,853.0,2033017.0,105201.0
4,Bhubaneswar,6.0,5.0,2404593.0,651825.0,824707.0,9249.0,230.0,777.0,2648793.0,7945.0


In [10]:
tier_1_2_cities = preprocessor.city_mapping['City'].values

# Economic Data Features

In [20]:
city_to_district_economic_data_mapping = dict(zip(preprocessor.city_mapping['City'], preprocessor.city_mapping['EconomicData_District']))
def get_economic_data(cities):
    cities_latest_gdps = []
    cities_gdp_history = []
    for city in cities:
        district = city_to_district_economic_data_mapping[city]
        if(pd.isnull(district)):
            cities_latest_gdps.append(np.nan)
            cities_gdp_history.append([])
        else:
            city_economic_data = preprocessor.economic_data[preprocessor.economic_data['District'] == district].sort_values("Year")
            city_latest_gdp = city_economic_data.iloc[city_economic_data.shape[0] - 1]['GDP']
            cities_latest_gdps.append(city_latest_gdp)
            city_gdp_history = city_economic_data['GDP'].values
            cities_gdp_history.append(city_gdp_history)
    economic_data = pd.DataFrame([cities, cities_latest_gdps, cities_gdp_history], index = ['City', 'GDP', 'GDP_history']).transpose().reset_index(drop = True)
    for col in economic_data.columns[1:-1]:
        economic_data[col] = economic_data[col].fillna(np.nanmean(economic_data[col].values))
    return economic_data

economic_data = get_economic_data(tier_1_2_cities)
economic_data.head()

,City,GDP,GDP_history
0,Ahmedabad,31638.670872,[]
1,Bengaluru,87206.000000,"[21876.93, 20490.12, 25285.1, 27768.66, 28290...."
2,Mumbai,191910.000000,"[54374.09, 52122.34, 55800.05, 59536.03, 65657..."
3,Pune,93562.000000,"[27231.84, 26229.79, 26871.17, 29131.93, 32026..."
4,Chennai,16670.560000,"[13215.12, 12725.53, 12460.29, 12812.12, 13831..."


In [21]:
def get_tourism_data(cities):
    
    def distance_factor(miles):
        return np.exp(miles / (-200))
    
    def closest_node(node, nodes):
        dist = np.sum((nodes - node)**2, axis=1)
        return np.argmin(dist)
    
    all_cities_coords = preprocessor.city_mapping['Airport_City_Coords']
    all_cities_coords_lat = all_cities_coords.apply(lambda x: float(x.split(',')[0])).values
    all_cities_coords_lon = all_cities_coords.apply(lambda x: float(x.split(',')[1])).values
    all_cities_coords = np.asarray(list(zip(all_cities_coords_lat, all_cities_coords_lon)))
    
    all_cities_tourism_dict = {}
    
    for idx, row in preprocessor.tourist_loc_coords_data.iterrows():
        tourist_loc_coord = np.asarray([row['Latitude'], row['Longitude']])
        closest_idx = closest_node(tourist_loc_coord, all_cities_coords)
        closest_city = cities[closest_idx]
        closest_coord = all_cities_coords[closest_idx]
        closest_dist = geopy.distance.geodesic(tourist_loc_coord, closest_coord).miles
        factor = distance_factor(closest_dist)
        tourism_data = preprocessor.monument_visitors_data.loc[idx].values
        if(closest_city in all_cities_tourism_dict):
            all_cities_tourism_dict[closest_city] += tourism_data[1:] * factor
        else:
            all_cities_tourism_dict[closest_city] = tourism_data[1:] * factor
    tourism_data = pd.DataFrame.from_dict(all_cities_tourism_dict, orient = 'index', columns = preprocessor.monument_visitors_data.columns[1:]).reset_index(drop = False).rename({'index': 'City'}, axis = 1)
    return tourism_data
    
tourism_data = get_tourism_data(tier_1_2_cities)
tourism_data.head()

,City,NumMonuments2020,NumMonuments2022,Domestic2019-20,Domestic2020-21,Domestic2021-22,Foreign2019-20,Foreign2020-21,Foreign2021-22,Domestic2018-19,Foreign2018-19
0,Gurugram,5.273227,5.273227,4.472232e+06,1.207976e+06,3.052370e+06,817136.264492,8260.802427,35205.232615,5.581264e+06,1.070223e+06
1,Pune,5.419323,5.419323,2.726150e+06,6.529268e+05,1.241964e+06,38111.232378,465.037828,2055.277359,2.638435e+06,4.522266e+04
2,Bengaluru,7.399579,7.399579,1.694443e+06,4.800086e+05,7.634647e+05,67012.546159,2056.047495,15870.326209,1.899249e+06,8.779623e+04
3,Bhopal,16.666635,13.742239,2.326419e+06,8.036434e+05,1.542882e+06,60905.889836,670.914791,1284.662472,2.522180e+06,1.035344e+05
4,Bhubaneswar,5.903807,4.919839,2.366042e+06,6.413749e+05,8.114852e+05,9100.718942,226.312613,764.543044,2.606327e+06,7.817625e+03
